In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from src.data_structures import Instance
from src.data_structures.features import *
from src.solvers.DLHEU2 import DHEU
from src.solvers.ActorCritic.solver import ZeroReductor2,ActorCritic
from pathlib import Path
from src.solvers.collection import SolverCollection,SolverConfig
from matplotlib import pyplot as plt

In [2]:
path = Path("actor_critic_data")
instances = ( Instance.generate(500,20) for i in range(1000))
instances = iter(instances)

In [3]:
#Primero entreno inicialmente al critico
ac = ActorCritic(8,2)
optimizer = optim.Adam(ac.parameters(),lr=0.01)
criterion = nn.L1Loss()
graficar = lambda lista: plt.plot([i for i in range(len(lista))],[i for i in lista])

In [4]:
losses = []
for i in range(1000):
    epoch_losses = []
    sols = []
    predicted_metrics = []
    metrics = []
    instance = Instance.generate(1000,10)
    actor_loss = 0
    critic_loss = 0
    threshold = 0.1
    max_step = 100
    max_repeat = 5
    zr = ZeroReductor2(instance)
    eq_th = 1e-6
    done = False
    #Calculo la mejor solucion para la instancia
    best_sol = SolverCollection.gurobi(instance,SolverConfig.optimal())
    while not done:
        #Resuelvo con el estado inicial
        real_value = zr.solve()
        sols.append(real_value)
        guess_actions, predicted_metric = ac(state)
        if (best_sol.o-real_value.o) < eq_th:
            es_el_optimo = 1
        else:
            es_el_optimo = 0
        loss = criterion(torch.tensor(es_el_optimo),predicted_metric)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        old_n_items = zr.instance.n_items
        zr.step(threshold = threshold, max_step=max_step)
        if zr.instance.n_items != old_n_items:
            stuck = 0
        else:
            stuck += 1
        if torch.min(zr.actual_pred) > 0.5 or stuck > 1:
            done = True
        epoch_losses.append(loss.item())
        print(f"Best: {best_sol.o} Current: {real_value.o} Equal: {(best_sol.o-real_value.o) < eq_th} Predicted: {predicted_metric}")
        avg_loss = torch.mean(torch.tensor(epoch_losses))
    plt.clf()
    losses.append(avg_loss)
    graficar(losses)
    plt.savefig("Criticloss.png")


Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-21


/usr/lib/python3.11/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Best: 48476.37910866037 Current: 48476.37910866039 Equal: True Predicted: tensor([0.5143], grad_fn=<SigmoidBackward0>)
Best: 48476.37910866037 Current: 48424.54401623236 Equal: False Predicted: tensor([0.5894], grad_fn=<SigmoidBackward0>)
Best: 48476.37910866037 Current: 48369.644704622384 Equal: False Predicted: tensor([0.5796], grad_fn=<SigmoidBackward0>)
Best: 48476.37910866037 Current: 48228.63467226655 Equal: False Predicted: tensor([0.5487], grad_fn=<SigmoidBackward0>)
Best: 48476.37910866037 Current: 48042.56195194027 Equal: False Predicted: tensor([0.5092], grad_fn=<SigmoidBackward0>)
Best: 48476.37910866037 Current: 48042.56195194027 Equal: False Predicted: tensor([0.4675], grad_fn=<SigmoidBackward0>)
Best: 39572.18342122357 Current: 39572.18342122358 Equal: True Predicted: tensor([0.4611], grad_fn=<SigmoidBackward0>)
Best: 39572.18342122357 Current: 39561.106955490286 Equal: False Predicted: tensor([0.4287], grad_fn=<SigmoidBackward0>)
Best: 39572.18342122357 Current: 39555.9